<h1 style='color:Green'>Baseline Modeling</h1>
<h3>📌 Objectives </h3>
<pre>
  Establish simple, interpretable benchmark models to:
  - Validate feature quality
  - Create RMSLE reference scores
  - Compare against advanced ML/DL later

 No heavy tuning, no leakage, no deep learning.
</pre>

<h2 style='color:Green'>Load Feature Data</h2>
<h3>🎯 Goal </h3>
<pre>
  - Load engineered features
  - Drop only lag/rolling NaNs
  - Keep time order intact
</pre>

<h3>Clone GitHub Repository</h3>

In [1]:
# Clone GitHub Repository
!git clone https://github.com/sabin74/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform.git

Cloning into 'Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 48 (delta 15), reused 26 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 21.62 MiB | 44.73 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Filtering content: 100% (11/11), 315.52 MiB | 66.13 MiB/s, done.


## Set Project Root

This keeps paths identical to local setup.

In [2]:
# Set  PROJECT_ROOT
import os

PROJECT_ROOT = "/content/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform"
os.chdir(PROJECT_ROOT)

print("Current directory:", os.getcwd())


Current directory: /content/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform


## Load Feature Data
🎯 Goal
 - Load engineered data
 - Drop only lag/rolling NaNs
 - Preserve time order
 - Avoid memory waste

In [3]:
# Import and Load Data
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")
from pathlib import Path

In [4]:
# Load Feature Dataset from Repo
FEATURE_DATA = Path("data/features")

train = pd.read_parquet(FEATURE_DATA / "train_features.parquet")


In [5]:
# Convert Date and Sort
train['date'] = pd.to_datetime(train['date'])

train = train.sort_values(
    ['store_nbr', 'family', 'date']
).reset_index(drop=True)

In [6]:
# Identify Lag and Rolling Columns
lag_roll_cols = [
    col for col in train.columns
    if ("lag" in col) or ("roll" in col)
]

In [7]:
# Drop NaN in lag/rolling features
train_model = train.dropna(
    subset = lag_roll_cols
).reset_index(drop=True)

## Define Target & Feature Sets
### Target Options
  - sales_log (preferred for RMSLE)
###Feature Sets
  - Time features
  - Lag features
  - Rolling features
  - Promotions
  - Oil
  - Encodings

In [8]:
# Targe Variable
TARGET = 'sales_log'

In [9]:
# Define Feature Groups
TIME_FEATURES = [
    "day", "week_of_year", "monty", "year",
    "day_of_week", "is_weekend", "is_payday"
]

LAG_FEATURES = [
    "sales_lag_1", "sales_lag_7",
    "sales_lag_14", "sales_lag_28"
]

ROLL_FEATURES = [
    "sales_roll_mean_7", "sales_roll_mean_14", "sales_roll_mean_28",
    "sales_roll_std_7", "sales_roll_std_14", "sales_roll_std_28"
]

PROMO_FEATURES = [
    "onpromotion",
    "promo_lag_1", "promo_lag_7",
    "promo_freq_7", "promo_freq_14", "promo_freq_28",
    "promo_roll_sum_7", "promo_roll_sum_14", "promo_roll_sum_28"
]

OIL_FEATURES = [
    "dcoilwtico", "oil_lag_7", "oil_lag_14", "oil_lag_28"
]

ENCODING_FEATURES = [
    "store_te", "family_te"
]


In [10]:
# Final Feature List
FEATURES = (
    TIME_FEATURES + LAG_FEATURES + ROLL_FEATURES +
    PROMO_FEATURES + OIL_FEATURES + ENCODING_FEATURES
)

In [11]:
# Check which features actually exist
print(f"Total features in list: {len(FEATURES)}")
print(f"Features missing from train_model:")
missing_features = [f for f in FEATURES if f not in train_model.columns]
for f in missing_features:
    print(f"  - {f}")

Total features in list: 32
Features missing from train_model:


In [12]:
# Preparing Modeling Metrics
X = train_model[FEATURES]
y = train_model[TARGET]

## Time-Based Validation Strategy
**Why**: Time series ≠ random split
### Validation Method
  - Last N days as validation
  - Example:
    - Train: up to 2017-07-15
    - Valid: 2017-07-16 → 2017-08-15

In [13]:
# Define Validation Cutoff Date
TRAIN_END_DATE = "2017-07-15"

In [14]:
# Create Time Based Split
train_mask = train_model['date'] <= TRAIN_END_DATE
valid_mask = train_model['date'] > TRAIN_END_DATE

X_train = X[train_mask]
y_train = y[train_mask]

X_valid = X[valid_mask]
y_valid = y[valid_mask]

In [15]:
# Also keep the full train_model rows for validation (for accessing lag columns)
train_valid = train_model[valid_mask].reset_index(drop=True)

print("DATA SPLIT SUMMARY:\n")
print(f"Train dates: {train_model[train_mask]['date'].min().date()} to {train_model[train_mask]['date'].max().date()}")
print(f"Validation dates: {train_model[valid_mask]['date'].min().date()} to {train_model[valid_mask]['date'].max().date()}")
print(f"Train samples: {len(X_train):,}")
print(f"Validation samples: {len(X_valid):,}")

DATA SPLIT SUMMARY:

Train dates: 2013-01-29 to 2017-07-15
Validation dates: 2017-07-16 to 2017-08-15
Train samples: 2,949,210
Validation samples: 55,242


## BASELINE MODEL 1 – NAIVE & MOVING AVERAGE
### Goal
  - Establish simple benchmark scores
  - Validate feature correctness
  - Create RMSLE reference

In [16]:
## Define Evaluation Metrics (RMSLE)
def rmsle(y_true, y_pred):
    """
    Root Mean Squared Log Error
    """
    y_true = np.maximum(0, y_true)
    y_pred = np.maximum(0, y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))



In [17]:
# Prepare Validation Ground Truth
y_valid_true = np.expm1(y_valid)

In [18]:
# Extract Prediction (log-space)
naive_pred_log = X_valid["sales_lag_1"]


In [19]:
# Convert Back to Original Scale
naive_pred = np.expm1(naive_pred_log)


In [20]:
# Define a safe inverse transform
def safe_expm1(x, clip_max=1):
    """
    Safely inverse log transform:
    - clips extreme log values
    - prevents overflow
    """
    x = np.clip(x, a_min=None, a_max=clip_max)
    return np.expm1(x)


In [21]:
# Evaluate RMSLE
naive_pred_log = X_valid["sales_lag_1"]
naive_pred = safe_expm1(naive_pred_log)

naive_rmsle = rmsle(y_valid_true, naive_pred)

print(f"Naive Forecast RMSLE: {naive_rmsle:.5f}")


Naive Forecast RMSLE: 3.62982


## BASELINE MODEL 1 – MOVING AVERAGE
### Predict sales using rolling mean of past sales\

In [22]:
# 7 day, 14 day & 28 day Average
ma7_pred = safe_expm1(X_valid["sales_roll_mean_7"])
ma14_pred = safe_expm1(X_valid["sales_roll_mean_14"])
ma28_pred = safe_expm1(X_valid["sales_roll_mean_28"])

ma7_rmsle = rmsle(y_valid_true, ma7_pred)
ma14_rmsle = rmsle(y_valid_true, ma14_pred)
ma28_rmsle = rmsle(y_valid_true, ma28_pred)

print(f"MA-7 RMSLE:  {ma7_rmsle:.5f}")
print(f"MA-14 RMSLE: {ma14_rmsle:.5f}")
print(f"MA-28 RMSLE: {ma28_rmsle:.5f}")


MA-7 RMSLE:  3.62609
MA-14 RMSLE: 3.62613
MA-28 RMSLE: 3.62632


## Baseline Model 3: Linear Regression
 - Penalizes large coefficients
 - Handles multicollinearity
 - Usually better than plain Linear

In [23]:
# Scaling data using Standard Scaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [24]:
# Initialize Regression Model
lr = LinearRegression(n_jobs=-1)
lr.fit(X_train_scaled, y_train)

LinearRegression(n_jobs=-1)

In [25]:
# Predict and Inverse Transform
lr_pred_log = lr.predict(X_valid_scaled)
lr_pred = np.expm1(lr_pred_log)
y_valid_true = np.expm1(y_valid)

In [26]:
# Calculate RMSLE
lr_rmsle = rmsle(y_valid_true, lr_pred)

print(f"Linear Regression RMSLE: {lr_rmsle:.5f}")

Linear Regression RMSLE: 0.9663


Baseline Model 4 – Ridge Regression
 - Penalizes large coefficients
 - Handles multicollinearity
 - Usually better than plain Linear

In [27]:
# Initialize Ridge
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)

Ridge()

In [28]:
# Predict and Inverse Transform
ridge_pred_log = ridge.predict(X_valid_scaled)
ridge_pred = np.expm1(ridge_pred_log)

In [29]:
# Calculate RMSLE
ridge_rmsle = rmsle(y_valid_true, ridge_pred)

print(f"Ridge Regression RMSLE: {ridge_rmsle:.5f}")

Ridge Regression RMSLE: 0.9663


Baseline Model 4 – Lasso Regression
 - Feature selection
 - Sparsity
 - Slightly worse than Ridge but interpretable

In [30]:
# Initialize Lasso
lasso = Lasso(alpha=0.0005, max_iter=3000)
lasso.fit(X_train_scaled, y_train)

Lasso(alpha=0.0005, max_iter=3000)

In [31]:
# Predict and Inverse Transform
lasso_pred_log = lasso.predict(X_valid_scaled)
lasso_pred = np.expm1(lasso_pred_log)

In [32]:

# Calculate RMSLE
lasso_rmsle = rmsle(y_valid_true, lasso_pred)

print(f"Lasso Regression RMSLE: {lasso_rmsle:.5f}")

Lasso Regression RMSLE: 0.9661


### Baseline Comparison Summary

In [39]:
baseline_results = pd.DataFrame({
    "Model": [
        "Naive",
        "Moving Average (7)",
        "Moving Average (14)",
        "Moving Average (28)",
        "Linear Regression",
        "Ridge Regression",
        "Lasso Regression"
    ],
    "RMSLE": [
        naive_rmsle,
        ma7_rmsle,
        ma14_rmsle,
        ma28_rmsle,
        lr_rmsle,
        ridge_rmsle,
        lasso_rmsle
    ]
}).sort_values("RMSLE")

baseline_results


NameError: name 'lasso_rmsle' is not defined